In [0]:
from pyspark.sql import functions as F


In [0]:
# Load csv file into dataframe and convert into parquet format 

df_oct = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",header=True,inferSchema=True)
df_nov = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",header=True,inferSchema=True)

df_oct.write.mode("overwrite").parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_nov.write.mode("overwrite").parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/nov/")



In [0]:
# read teh parquest type files
df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_nov = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/nov/")

In [0]:
display(df_oct.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-13T06:25:46.000Z,view,1002544,2053013555631882655,electronics.smartphone,apple,460.51,518958788,e7e27c5c-1e78-4812-9f55-cdc658bb40fe
2019-10-13T06:25:46.000Z,view,3700301,2053013565983425517,appliances.environment.vacuum,vitek,120.93,557977070,7afc206c-7259-4beb-b49e-4662267481e9
2019-10-13T06:25:46.000Z,view,49100004,2127425375913902544,null,null,45.05,514456508,9d6837a5-40df-49de-ba3c-5625f2659230
2019-10-13T06:25:46.000Z,view,9200409,2053013552913973497,computers.peripherals.keyboard,defender,12.56,512530774,df2d048d-c1ae-41bf-b5c3-348617f505a8
2019-10-13T06:25:46.000Z,view,1306558,2053013558920217191,computers.notebook,acer,1801.82,523366823,0c7f0449-74d5-4b0c-bb07-ba04cc7a6681


In [0]:
df_oct.select("event_type").distinct().show()

+----------+
|event_type|
+----------+
|  purchase|
|      cart|
|      view|
+----------+



In [0]:
df_oct.select("brand","event_type","price","category_code") \
.filter((F.col("event_type") == "purchase") & 
(F.col("brand") == "apple") &
(F.col("category_code") == "electronics.smartphone")).show(5,truncate=False)

+-----+----------+-------+----------------------+
|brand|event_type|price  |category_code         |
+-----+----------+-------+----------------------+
|apple|purchase  |358.57 |electronics.smartphone|
|apple|purchase  |952.15 |electronics.smartphone|
|apple|purchase  |1003.85|electronics.smartphone|
|apple|purchase  |696.13 |electronics.smartphone|
|apple|purchase  |952.15 |electronics.smartphone|
+-----+----------+-------+----------------------+
only showing top 5 rows


In [0]:
df_oct.select("event_type","brand") \
.where(F.col("event_type").isin ("view","purchase") & F.col("brand").isin ("acer","apple")) \
.groupby ("event_type","brand").count().orderBy("event_type","brand").show()

+----------+-----+-------+
|event_type|brand|  count|
+----------+-----+-------+
|  purchase| acer|   6882|
|  purchase|apple| 142873|
|      view| acer| 411871|
|      view|apple|3770597|
+----------+-----+-------+



In [0]:
#top 5 purchase of apple brand smartphone
df_oct.select("event_time","brand","event_type","price","category_code") \
.filter((F.col("event_type") == "purchase")  
      & (F.col("brand") == "apple") \
      & (F.col("category_code") == "electronics.smartphone")) \
.orderBy(F.desc("price")) \
.limit(5).display()

event_time,brand,event_type,price,category_code
2019-10-11T10:10:35.000Z,apple,purchase,2110.45,electronics.smartphone
2019-10-11T09:26:09.000Z,apple,purchase,2110.45,electronics.smartphone
2019-10-11T09:33:08.000Z,apple,purchase,2110.45,electronics.smartphone
2019-10-11T08:50:39.000Z,apple,purchase,2110.45,electronics.smartphone
2019-10-11T12:11:04.000Z,apple,purchase,2110.45,electronics.smartphone


In [0]:
# create a subset of data 
subsetdf= df_oct.select("event_time","brand","event_type","price","category_code") \
    .filter((F.col("event_type") == "view") &
    (F.col("brand") == "acer") &
    (F.col("category_code") == "computers.notebook")).orderBy(F.col("price").desc())


display(subsetdf)
subsetdf.write.mode("overwrite").saveAsTable("ecommerce.acer_view_event")



event_time,brand,event_type,price,category_code
2019-10-26T09:58:37.000Z,acer,view,2574.04,computers.notebook
2019-10-19T07:50:26.000Z,acer,view,2574.04,computers.notebook
2019-10-16T11:59:42.000Z,acer,view,2574.04,computers.notebook
2019-10-16T12:10:06.000Z,acer,view,2574.04,computers.notebook
2019-10-02T13:14:19.000Z,acer,view,2574.04,computers.notebook
2019-10-18T05:03:26.000Z,acer,view,2574.04,computers.notebook
2019-10-18T05:03:30.000Z,acer,view,2574.04,computers.notebook
2019-10-29T10:32:18.000Z,acer,view,2574.04,computers.notebook
2019-10-26T04:41:20.000Z,acer,view,2574.04,computers.notebook
2019-10-17T11:42:57.000Z,acer,view,2574.04,computers.notebook


In [0]:
%sql
SELECT * FROM ecommerce.acer_view_event LIMIT 20;


event_time,brand,event_type,price,category_code
2019-10-19T07:50:26.000Z,acer,view,2574.04,computers.notebook
2019-10-17T11:42:57.000Z,acer,view,2574.04,computers.notebook
2019-10-17T11:44:31.000Z,acer,view,2574.04,computers.notebook
2019-10-16T11:59:42.000Z,acer,view,2574.04,computers.notebook
2019-10-16T12:07:25.000Z,acer,view,2574.04,computers.notebook
2019-10-16T12:10:06.000Z,acer,view,2574.04,computers.notebook
2019-10-26T04:41:20.000Z,acer,view,2574.04,computers.notebook
2019-10-29T10:28:03.000Z,acer,view,2574.04,computers.notebook
2019-10-29T10:32:18.000Z,acer,view,2574.04,computers.notebook
2019-10-26T09:58:37.000Z,acer,view,2574.04,computers.notebook
